The Domain

The main goal is to generate a Matchcard. Sample Matchcards can be found in the Excel Sheet. A Team's Matchcard contains these informations:
* The Treasury of the team before a game
* The Winnings of the team in the match
* The upkeep, which is depending on whether or not players are Starplayers and how high the initial cost of the starplayer was
* The future treasury, which is the treasury before the game + winnings - upkeep
* Also, in case the coach has players with special roles, special bowlbot commands are generated.

In [1]:
type Currency = Currency of int
let sumCurrency cs = Currency(List.sumBy (fun (Currency c) -> c) cs)

type Position =
    | Shaman
    | PlayerCoach
    | Cheerleader
    | Filcher
    | BribedRef
    | Fan
    | SquigTamer

let toPosition p = 
    match p with
    | "Shaman" -> Some Shaman
    | "Player-Coach" -> Some PlayerCoach
    | "Cheerleader" -> Some Cheerleader
    | "Filcher" -> Some Filcher
    | "Bribed Ref" -> Some BribedRef
    | "Fan" -> Some Fan
    | "Squig Tamer" -> Some SquigTamer
    | _ -> None

let toUpkeep c =
    if c < 56000 then (Currency 10000)
    elif c < 106000 then (Currency 20000)
    elif c < 156000 then (Currency 30000)
    elif c < 206000 then (Currency 40000)
    elif c < 301000 then (Currency 50000)
    else (Currency 60000)

type Player = 
    | Normal of string
    | Star of string * Currency
    | Special of string * Position

let rec getBowlbotMessage position players =
    match players with
    | [] -> "Not rostered"
    | head::tail ->
        match head with
        | Special (_,p) when p = position -> 
            match p with
            | Shaman -> "bb1d6,1d8"
            | PlayerCoach -> "bb1dX,1d15"
            | Cheerleader -> "bb1d6"
            | Filcher -> "bb1d6"
            | BribedRef -> "bb1d6"
            | Fan -> "bb1d6"
            | SquigTamer -> "bb1d6"
        | _ -> getBowlbotMessage position tail

type FanFactor = FanFactor of int

type Team = {
    Name : string
    Treasury : Currency
    FanFactor : FanFactor
    Players : Player list
}
let createTeam name treasury fanfactor players getUpkeep =
    {
        Name = name
        Treasury = treasury
        FanFactor = fanfactor
        Players =
            players 
            |> List.map (fun (name, position, positionid) -> 
                if position.Equals(name) then 
                    Star (name, getUpkeep positionid)
                else
                    match toPosition position with
                    | Some pos -> Special (name, pos)
                    | None -> Normal name
                )
    }

type MatchResult = {
    Winnings : Currency
}

type Match = {
    Team1 : Team * MatchResult
    Team2 : Team * MatchResult
}

let toMatchCard (t,r) =
    let teamname = t.Name
    let (Currency winnings) = r.Winnings
    let (Currency treasury) = t.Treasury
    let (Currency upkeep) = 
        t.Players 
        |> List.choose (fun p -> 
            match p with
            | Star (_,upkeep) -> Some upkeep
            | _ -> None)
        |> sumCurrency
        
    $"{teamname}\n" +
    $"XXXLBowl #1\n" +
    $"Treasury: {treasury-winnings}\n"+
    $":winnings: {winnings}\n" +
    $":ref: {getBowlbotMessage BribedRef t.Players}\n" +
    $":fan: {getBowlbotMessage Fan t.Players}\n" +
    $":upkeep: {upkeep}\n" +
    $"Future Treasury: {treasury-upkeep}\n" +
    $"Specialist Rolls:\n" +
    $":cheer: {getBowlbotMessage Cheerleader t.Players}\n" +
    $":fan: {getBowlbotMessage Fan t.Players}\n" +
    $":filcher: {getBowlbotMessage Filcher t.Players}\n" +
    $":coach: {getBowlbotMessage PlayerCoach t.Players}\n" +
    $":shaman: {getBowlbotMessage Shaman t.Players}\n" +
    $":tamer: {getBowlbotMessage SquigTamer t.Players}\n"

The API
https://fumbbl.com/apidoc/#

In [1]:
#r "nuget: FsHttp"

open FsHttp
open FSharp.Data

[<Literal>] 
let FumblAPI = "https://fumbbl.com/api/"
[<Literal>] 
let PositionEndpoint = FumblAPI + "position/get/"
[<Literal>] 
let TeamEndpoint = FumblAPI + "team/get/"
[<Literal>] 
let MatchEndpoint = FumblAPI + "match/get/"

[<Literal>] 
let PositionSample = PositionEndpoint + "36290"
type JsonPosition = JsonProvider<PositionSample>
let getPosition (i:int) = 
    http {
        GET $"{PositionEndpoint}{i}"
    }
    |> toText
    |> JsonPosition.Parse

let getUpkeep i = 
    getPosition i
    |> fun pos -> toUpkeep pos.Cost

[<Literal>] 
let TeamSample = TeamEndpoint + "997349"
type JsonTeam = JsonProvider<TeamSample>
let getTeam (i:int) =
    http {
        GET $"{TeamEndpoint}{i}"
    }
    |> toText
    |> JsonTeam.Parse
    |> fun t -> createTeam t.Name (Currency t.Treasury) (FanFactor t.FanFactor) (t.Players |> Array.toList |> List.map (fun p -> (p.Name, p.Position, p.PositionId))) getUpkeep

[<Literal>] 
let MatchSample = MatchEndpoint + "4284928"
type JsonMatch = JsonProvider<MatchSample>
let getMatch (i:int) = 
    http {
        GET $"{MatchEndpoint}{i}"
    }
    |> toText
    |> JsonMatch.Parse
    |> fun m -> { 
            Team1 = (getTeam m.Team1.Id, {Winnings = Currency m.Team1.Winnings})
            Team2 = (getTeam m.Team2.Id, {Winnings = Currency m.Team2.Winnings})
        }

To get the required data to create MatchCards for a Match first, load the match with 
```
getMatch {MatchID}
```

In [1]:
let matchData = getMatch 4284928

Then you can create matchcards for each team with
```
toMatchCard {team}
```

In [1]:
toMatchCard matchData.Team1

Bogan Picnic Hamper Stealers
XXXLBowl #1
Treasury: 144000
:winnings: 60000
:ref: Not rostered
:fan: Not rostered
:upkeep: 0
Future Treasury: 204000
Specialist Rolls:
:cheer: Not rostered
:fan: Not rostered
:filcher: bb1d6
:coach: Not rostered
:shaman: Not rostered
:tamer: Not rostered


In [1]:
toMatchCard matchData.Team2

Black Tooth Rips
XXXLBowl #1
Treasury: 11000
:winnings: 70000
:ref: bb1d6
:fan: bb1d6
:upkeep: 70000
Future Treasury: 11000
Specialist Rolls:
:cheer: bb1d6
:fan: bb1d6
:filcher: bb1d6
:coach: Not rostered
:shaman: bb1d6,1d8
:tamer: Not rostered
